In [1]:
import proofer

In [2]:
from imp import reload
reload(proofer)

<module 'proofer' from '/Users/caseyschneider-mizell/Work/allen_work/data_analysis/proofreading_state/proofer.py'>

In [3]:
datastack = 'minnie65_phase3_v1'

In [4]:
proofer.initial_state_url(datastack, return_as='html')

In [5]:
orig_url = "https://akhilesh-graphene-sharded-dot-neuromancer-seung-import.appspot.com/?json_url=https://globalv1.daf-apis.com/nglstate/api/v1/6218267215527936"

In [6]:
orig_url = "https://akhilesh-graphene-sharded-dot-neuromancer-seung-import.appspot.com/?json_url=https://globalv1.daf-apis.com/nglstate/api/v1/6085909476802560"

In [7]:
proofer.build_proofreading_state(orig_url, datastack, render_as='html')

100%|██████████| 1479755/1479755 [01:28<00:00, 16767.87it/s]


In [ ]:
import pdb
pdb.pm()

In [ ]:
from meshparty import meshwork, trimesh_io, trimesh_vtk, mesh_filters
from annotationframeworkclient.frameworkclient import FrameworkClient
import numpy as np
import pandas as pd
import nglui
import re

EP_TAGS = ['nat_end', 'unc_end', 'unc_cont']
BP_TAGS = ['checked']
EP_COLOR = '#BBCC33'
BP_COLOR = '#3322BB'

In [ ]:
client = FrameworkClient(datastack)

segs, annos = proofer.parse_state(orig_url, datastack, client)
sb = proofer.proofreading_statebuilder(client, segs)

In [ ]:
reload(proofer)

In [ ]:
df = proofer.get_topo_points(client, segs, annos, min_size=1000)

In [ ]:
import dill as pickle

with open('df.pickle', 'rb') as f:
    df = pickle.load(f)

In [ ]:
import dill as pickle

with open('df.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
sb.render_state(df, return_as='html', url_prefix=client.info.info_cache.get(datastack).get('viewer_site'))

In [ ]:
from meshparty import meshwork, trimesh_io, mesh_filters
from annotationframeworkclient import FrameworkClient

In [ ]:
client = FrameworkClient(datastack)
mm = trimesh_io.MeshMeta(cv_path=client.info.segmentation_source())
mesh = mm.mesh(seg_id=864691134917372426)

In [ ]:
mk = mesh_filters.filter_largest_component(mesh)
meshf = mesh.apply_mask(mk)
nrn = meshwork.Meshwork(mesh=meshf)

In [ ]:
soma_pt = np.array([240176, 205268, 21365]) * np.array([4,4,40])

In [ ]:
nrn.skeletonize_mesh(soma_pt=soma_pt, soma_thresh_distance=10000, compute_radius=False, overwrite=True)

In [ ]:
cps = nrn.skeleton.cover_paths
cp_ends = np.array([nrn.skeleton.parent_nodes(cp[-1]) for cp in cps])


In [ ]:
bps = nrn.skeleton.branch_points
cp_queue = [0]
primary_cps = [ii for ii, cp in enumerate(cp_ends) if cp == nrn.skeleton.root]

In [ ]:
primary_cps

In [ ]:
proofer.nglui.statebuilder.PointMapper?

In [ ]:
def _branch_and_end_points_ordered(nrn):
    cps = nrn.skeleton.cover_paths
    cp_ends = np.array([nrn.skeleton.parent_nodes(cp[-1]) for cp in cps])

    bps = nrn.skeleton.branch_points
    cp_queue = [0]
    primary_cps = [ii for ii, cp in enumerate(cp_ends) if cp == nrn.skeleton.root]
    cp_queue.extend(primary_cps)

    is_done = np.full(len(cp_ends), False)
    bp_groups = []
    eps = []
    while not np.all(is_done):
        print(cp_queue)

        ind_to_do = cp_queue.pop(0)
        is_done[ind_to_do] = True
        cp = cps[ind_to_do]

        # Get branch points
        bps_on_cp = bps[np.isin(bps, cp)]
        bp_groups.append(bps_on_cp)

        # Get end point
        eps.append(cp[0])

        # Get other cps to check
        cp_offshoots = np.flatnonzero(np.isin(cp_ends, cp))
        if len(cp_offshoots) > 0:
            cp_offshoots = cp_offshoots[~is_done[cp_offshoots]]
            lens = np.array([nrn.skeleton.distance_to_root[cps[ii][0]] -
                             nrn.skeleton.distance_to_root[cps[ii][-1]] for ii in cp_offshoots])
            cp_offshoots = cp_offshoots[np.argsort(lens)]
            for ind in cp_offshoots:
                if ind not in cp_queue:
                    cp_queue.insert(0, ind)

    bp_points = []
    for skinds in bp_groups:
        if len(skinds) > 0:
            bp_points.append(nrn.skeleton.vertices[skinds])
    bp_points = np.vstack(bp_points)

    ep_points = []
    for skind in eps:
        ep_points.append(nrn.skeleton.vertices[skind])
    ep_points = np.vstack(ep_points)

    return bp_points, ep_points


In [ ]:
_branch_and_end_points_ordered(nrn)

In [ ]:
from meshparty import trimesh_vtk

In [ ]:
ma = nrn.mesh_actor()
ska = nrn.skeleton_actor(line_width=4, color=(0,0,0))
trimesh_vtk.render_actors([ma,ska])

In [ ]:
cps = nrn.skeleton.cover_paths

cp_ends = np.array([nrn.skeleton.parent_nodes(cp[-1]) for cp in cps])

bps = nrn.skeleton.branch_points
cp_queue = [0]
is_done = np.full(len(cp_ends), False)
bp_groups = []
eps = []
while not np.all(is_done):
    ind_to_do = cp_queue.pop(0)
    print(cp_queue)
    is_done[ind_to_do] = True
    cp = cps[ind_to_do]
    
    # Get branch points
    bps_on_cp = bps[np.isin(bps, cp)]
    bp_groups.append(bps_on_cp)
    
    # Get end point
    eps.append(cp[0])
    
    # Get other cps to check
    cp_offshoots = np.flatnonzero(np.isin(cp_ends, cp))
    if len(cp_offshoots) > 0:
        cp_offshoots = cp_offshoots[~is_done[cp_offshoots]]
        lens = np.array([nrn.skeleton.distance_to_root[cps[ii][0]] - nrn.skeleton.distance_to_root[cps[ii][-1]] for ii in cp_offshoots])
        cp_offshoots = cp_offshoots[np.argsort(lens)]
        for ind in cp_offshoots:
            cp_queue.insert(0, ind)

In [ ]:
bp_points = []
for skinds in bp_groups:
    if len(skinds)>0:
        bp_points.append(nrn.vertices[skinds.to_mesh_region_point])

In [ ]:
bp_points = np.vstack(bp_points)

In [ ]:
ep_points = []
for skind in eps:
    ep_points.append(nrn.skeleton.vertices[skind])
ep_points = np.vstack(ep_points)

In [ ]:
ep_points

In [ ]:
bp_points

In [ ]:
bps = nrn.skeleton.branch_points

ind_to_do = 0
is_done = np.full(len(cp_ends), False)
bp_groups = []
eps = []

is_done[ind_to_do] = True
cp = cps[ind_to_do]

# Get branch points
bps_on_cp = bps[np.isin(bps, cp)]
bp_groups.append(bps_on_cp)


In [ ]:
cp_offshoots = np.flatnonzero(np.isin(cp_ends, cp))
cp_offshoots = cp_offshoots[np.invert(is_done[cp_offshoots])]


In [ ]:
cp_offshoots

In [ ]:
from nglui import nglite as neuroglancer

In [ ]:
from nglui import easyviewer, statebuilder

In [ ]:
easyviewer.annotation.group_annotations?

In [ ]:
from annotationframeworkclient import FrameworkClient

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

datastack = 'minnie65_phase3_v1'
client = FrameworkClient(datastack)

In [ ]:
img = statebuilder.ImageLayerConfig(client.info.image_source())
points = statebuilder.PointMapper(point_column='pt', group_column='grp')
anno = statebuilder.AnnotationLayerConfig(mapping_rules=[points])
sb = statebuilder.StateBuilder([img, anno])

In [ ]:
pts = np.random.randint(0, 10000, size=(10,3))
grp = np.random.randint(0,3, size=10).astype(np.float)
grp[[0,4]] = np.nan

In [ ]:
data = pd.DataFrame(data={'pt':pts.tolist(), 'grp':grp})

In [ ]:
statebuilder.AnnotationLayerConfig?

In [ ]:
sb.render_state(data)

In [ ]:
import pdb
pdb.pm()

In [ ]:
img = statebuilder.ImageLayerConfig

In [ ]:
pt1 = neuroglancer.PointAnnotation(point=[1,2,3], id='a1')
pt2 = neuroglancer.PointAnnotation(point=[2,3,4], id='a2')

In [ ]:
vwr = easyviewer.EasyViewer()

In [ ]:
vwr.add_image_layer('img', "graphene://https://minnie.microns-daf.com/proxy/minniev2_em/")
vwr.add_annotation_layer('anno')

In [ ]:
import numpy as np

In [ ]:
pt1 = easyviewer.annotation.point_annotation(np.array([2,3,4]))
pt2 = easyviewer.annotation.point_annotation([1,2,3])
grp = easyviewer.annotation.group_annotations([pt1, pt2], return_all=False)

In [ ]:
pt1.parent_id == grp.id

In [ ]:
vwr.add_annotations('anno', grp)

In [ ]:
pt1 = easyviewer.annotation.point_annotation(np.array([2,3,4]))
pt2 = easyviewer.annotation.point_annotation([1,2,3])
grp = neuroglancer.CollectionAnnotation(source=[1,2,3], entries=[pt1.id, pt2.id], id='1', children_visible=True)
vwr.add_annotations('anno', [grp, pt1, pt2])

In [ ]:
pt1.parent_id = '3'

In [ ]:
pt1

In [ ]:
vwr

In [ ]:
pt2 = easyviewer.annotation.line_annotation([1,2,3], [3,4,5])


In [ ]:
pt1.id = 3

In [ ]:
pt1.

In [ ]:
blah = np.array([1,2,3,np.nan, np.nan, 1, 1, 2, 3,3,2])

In [ ]:
vals, inv = np.unique(blah, return_inverse=True)

In [ ]:
np.flatnonzero(inv==0)

In [ ]:
np.flatnonzero(~np.isnan(vals))

In [ ]:
for i